# 📦 Library Imports and Environment Setup


In [1]:
import shapely
print(f"Using shapely {shapely.__version__}")

import copy
import math
import random
from concurrent.futures import ProcessPoolExecutor
from decimal import Decimal, getcontext

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from tqdm import tqdm


Using shapely 2.1.2


# ⚙️ Global Configuration and Precision Settings


In [2]:
getcontext().prec = 25
scale_factor = Decimal("1e15")

# 🎄 ChristmasTree Class Definition


In [3]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x="0", center_y="0", angle="0"):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal("0.0") * scale_factor, tip_y * scale_factor),
            (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

    def get_params(self):
        return self.center_x, self.center_y, self.angle

    def set_params(self, center_x, center_y, angle):
        self.__init__(str(center_x), str(center_y), str(angle))

# 🔧 Utility Functions for Scoring and Collision Detection

In [4]:
def calculate_score(trees):
    """Calculate bounding box score"""
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    return max(max_x - min_x, max_y - min_y) ** 2

def has_collision(trees):
    """Check for collisions between trees"""
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

# 🧠 Simulated Annealing for Small N (n < 20)

In [5]:
def get_random_initial_trees(n, max_attempts=100):
    """Generate random initial placement without collisions"""
    best_score = float("inf")
    best_trees = None
    grid_size = max(2.0, math.ceil(math.sqrt(n) * 0.8))

    for attempt in range(max_attempts):
        trees = []
        for _ in range(n):
            for _ in range(1000):
                x = random.uniform(-grid_size, grid_size)
                y = random.uniform(-grid_size, grid_size)
                angle = random.uniform(0, 360)
                tree = ChristmasTree(center_x=str(x), center_y=str(y), angle=str(angle))
                trees.append(tree)

                if has_collision(trees):
                    trees.pop()
                    continue
                else:
                    break

        if len(trees) == n:
            score = calculate_score(trees)
            if score < best_score:
                best_score = score
                best_trees = trees

    if best_trees is None:
        best_trees = []
        for i in range(n):
            x = (i % int(grid_size)) * 0.7
            y = (i // int(grid_size)) * 1.0
            tree = ChristmasTree(center_x=str(x), center_y=str(y), angle="0")
            best_trees.append(tree)

    return best_trees

def perturb_tree(tree, position_delta, angle_delta):
    """Perturb tree position and angle"""
    old_x, old_y, old_angle = tree.get_params()
    dx = Decimal(str(random.uniform(-position_delta, position_delta)))
    dy = Decimal(str(random.uniform(-position_delta, position_delta)))
    dangle = Decimal(str(random.uniform(-angle_delta, angle_delta)))
    new_x = old_x + dx
    new_y = old_y + dy
    new_angle = (old_angle + dangle) % 360
    tree.set_params(new_x, new_y, new_angle)
    return old_x, old_y, old_angle

def simulated_annealing(trees, T_init=2.0, T_min=0.001, alpha=0.95, 
                       iterations_per_temp=200, position_delta=0.1, angle_delta=30.0):
    """Simulated annealing optimization"""
    T = T_init
    current_trees = copy.deepcopy(trees)
    current_score = calculate_score(current_trees)
    best_trees = copy.deepcopy(current_trees)
    best_score = current_score

    while T > T_min:
        for _ in range(iterations_per_temp):
            new_trees = copy.deepcopy(current_trees)
            idx = random.randint(0, len(new_trees) - 1)
            tree = new_trees[idx]
            old_x, old_y, old_angle = perturb_tree(tree, position_delta, angle_delta)

            if has_collision(new_trees):
                tree.set_params(old_x, old_y, old_angle)
                continue

            new_score = calculate_score(new_trees)
            delta = new_score - current_score

            if delta < 0 or random.random() < math.exp(-delta / T):
                current_trees = new_trees
                current_score = new_score
                if new_score < best_score:
                    best_score = new_score
                    best_trees = copy.deepcopy(new_trees)
            else:
                tree.set_params(old_x, old_y, old_angle)

        T *= alpha

    return best_score, best_trees

def find_best_trees_annealing(n):
    """Simulated annealing approach for n < 20"""
    initial_trees = get_random_initial_trees(n, max_attempts=50)

    if n <= 10:
        T_init = 2.0
        iterations_per_temp = 200
        position_delta = 0.2
        angle_delta = 45.0
    else:
        T_init = 2.0
        iterations_per_temp = 300
        position_delta = 0.15
        angle_delta = 40.0

    improved_score, improved_trees = simulated_annealing(
        initial_trees, T_init=T_init, T_min=0.001, alpha=0.95,
        iterations_per_temp=iterations_per_temp,
        position_delta=position_delta, angle_delta=angle_delta
    )

    return improved_score, improved_trees

# 🔲 Grid Search for Large N (n >= 20)

In [6]:
def find_best_trees_grid_search(n):
    """Grid search approach for n >= 20"""
    best_score, best_trees = float("inf"), None

    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            all_trees = []
            rest = n
            r = 0

            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                rest -= m

                angle = 0 if r % 2 == 0 else 180
                x_offset = 0 if r % 2 == 0 else Decimal("0.7") / 2
                y = (r // 2 * Decimal("1.0") if r % 2 == 0 
                     else (Decimal("0.8") + (r - 1) // 2 * Decimal("1.0")))
                
                row_trees = [
                    ChristmasTree(center_x=Decimal("0.7") * i + x_offset, 
                                center_y=y, angle=angle)
                    for i in range(m)
                ]
                all_trees.extend(row_trees)
                r += 1

            score = calculate_score(all_trees)
            if score < best_score:
                best_score = score
                best_trees = all_trees

    return best_score, best_trees

# 🎯 Hybrid Solver (Combines SA + Grid Search)

In [7]:
def find_best_trees(n):
    """Hybrid approach: SA for n < 20, Grid Search for n >= 20"""
    if n < 20:
        return find_best_trees_annealing(n)
    else:
        return find_best_trees_grid_search(n)

# 🎲 Ensemble Solver with Multiple Seeds

In [8]:
def solve_for_n_with_ensemble(n, seeds=(0, 2025, 9999)):
    """Run hybrid solver with different seeds and pick best"""
    model_solutions = []
    scores = []

    for seed in seeds:
        random.seed(seed + n)
        np.random.seed(seed + n)

        score, trees = find_best_trees(n)
        model_solutions.append((score, trees))
        scores.append(score)

    scores = np.array(scores, dtype=float)
    best_idx = int(np.argmin(scores))
    best_score, best_trees = model_solutions[best_idx]
    
    if n in (1, 10, 25, 50, 100, 150, 200):
        print(f"n={n:3d} | best_score={best_score:.6f}")
    
    return best_score, best_trees

# 🚀 Main Computation Loop for N=1 to 200

In [9]:
ensemble_solutions = []

for n in tqdm(range(1, 201), desc="Computing solutions"):
    score, trees = solve_for_n_with_ensemble(n)
    ensemble_solutions.append((score, trees))

Computing solutions:   0%|          | 1/200 [00:46<2:34:47, 46.67s/it]

n=  1 | best_score=0.661253


Computing solutions:   5%|▌         | 10/200 [14:55<5:56:50, 112.69s/it]

n= 10 | best_score=4.955544


Computing solutions:  12%|█▎        | 25/200 [1:01:21<2:02:51, 42.12s/it]

...

n=100 | best_score=39.690000


Computing solutions:  75%|███████▌  | 150/200 [1:27:26<24:50, 29.80s/it]

n=150 | best_score=64.000000


Computing solutions: 100%|██████████| 200/200 [2:03:03<00:00, 36.92s/it]

n=200 | best_score=81.000000


# 📊 Calculate Overall Score

In [10]:
overall_score = sum(score / n for n, (score, _) in enumerate(ensemble_solutions, 1))
print(f"\n✅ Ensemble overall score: {overall_score:.6f}")


✅ Ensemble overall score: 85.918995


# 📝 Format and Export Submission CSV

In [11]:
def to_str(x):
    return f"s{round(float(x), 6)}"

rows = []
for n, (_, all_trees) in enumerate(ensemble_solutions, 1):
    assert len(all_trees) == n
    for i_t, tree in enumerate(all_trees):
        rows.append({
            "id": f"{n:03d}_{i_t}",
            "x": to_str(tree.center_x),
            "y": to_str(tree.center_y),
            "deg": to_str(tree.angle),
        })

df = pd.DataFrame(rows)
df.to_csv("submission.csv", index=False)
print(f"✅ submission.csv saved with {len(df)} rows")

✅ submission.csv saved with 20100 rows


# 👀 Preview Submission

In [12]:
df.head(20)

,id,x,y,deg
0,001_0,s-9.146226,s-0.12832,s-224.999647
1,002_0,s0.304461,s2.038269,s156.021729
2,002_1,s0.615807,s1.501107,s-23.140328
3,003_0,s1.813142,s0.999474,s109.935311
4,003_1,s1.339721,s1.405405,s155.403367
5,003_2,s1.944214,s1.460297,s62.196012
6,004_0,s-2.362982,s0.587065,s206.54873
7,004_1,s-1.679805,s0.591878,s164.288934
8,004_2,s-1.532529,s-0.227414,s114.912299
9,004_3,s-2.323341,s-0.313839,s27.268105
